<a href="https://colab.research.google.com/github/drajwer/defensegan/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
CONFIG_FILE = 'experiments/cfgs/gans/mnist.yml'
DATASETS = ['mnist']

In [2]:
!git clone https://github.com/drajwer/defensegan.git
!cd defensegan
%cd defensegan

Cloning into 'defensegan'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 400 (delta 0), reused 3 (delta 0), pack-reused 395
Receiving objects: 100% (400/400), 2.77 MiB | 2.44 MiB/s, done.
Resolving deltas: 100% (120/120), done.
/content/defensegan


In [10]:
from IPython.display import clear_output

!pip install -Iv tensorflow==1.15.0
!pip install -r requirements.txt

Streaming output truncated to the last 5000 lines.
  Found link https://files.pythonhosted.org/packages/20/51/71f2d77f3af6e95a6ac920425d54233c898cb04a2a2a45faa488f0562d9d/numpy-1.6.2.tar.gz#sha256=0b83d96ab79711b548470b7aeee1272c4ba8fcbba6586a67116b566a21ed16c6 (from https://pypi.org/simple/numpy/), version: 1.6.2
  Skipping link: unsupported archive format: .exe: https://files.pythonhosted.org/packages/a2/fb/79ad4e77725c64e19aca63f431ceaf59a84251229fd66edf21099cb73bac/numpy-1.6.2.win32-py2.5.exe#sha256=e5e517ff400f1d25c094abad5bd8c8a41dda3e98236ef4a065a02b24f3f51a5c (from https://pypi.org/simple/numpy/)
  Skipping link: unsupported archive format: .exe: https://files.pythonhosted.org/packages/f2/3c/6b5157c028f834c925871cc2a30ff54368ae5f5e2ad2c45bedf22a85b23a/numpy-1.6.2.win32-py2.6.exe#sha256=ba3fdba01264ac57c0228e165b9eec2904a6135a7ba73ba4b345b1238241007e (from https://pypi.org/simple/numpy/)
  Skipping link: unsupported archive format: .exe: https://files.pythonhosted.org/packages/9

     |████████████████████████████████| 12.1MB 262kB/s 
     |████████████████████████████████| 46.7MB 80kB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 337kB 45.3MB/s 
     |████████████████████████████████| 24.9MB 70.5MB/s 
     |████████████████████████████████| 35.2MB 172kB/s 
     |████████████████████████████████| 15.0MB 308kB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
ERROR: dopamine-rl 1.0.5 has requirement opencv-python>=3.4.1.15, but you'll have opencv-python 3.4.0.12 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.20.0 which is incompatible.
ERROR: blis 0.2.4 has requirement numpy>=1.15.0, but you'll have numpy 1.14.2 which is incompatible.
ERROR: fastai 0.7.0 has requirement torch<0.4, but you'll have torch 1.4.0 which is incompati

In [4]:
import argparse
import sys

import tensorflow as tf

from models.gan import MnistDefenseGAN, FmnistDefenseDefenseGAN, \
    CelebADefenseGAN
from utils.config import load_config

tflib/plot.py:6: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 499, in start
    self.io_loop.start()
  File "/usr/local/lib/python2.7/dist-packages/tornado/ioloop.py", line 888, in start
    h

In [5]:
from __future__ import print_function
import os
import sys
import json
import zipfile
import argparse
import requests
import subprocess
from tqdm import tqdm
from six.moves import urllib

parser = argparse.ArgumentParser(description='Download dataset for DCGAN.')
parser.add_argument('datasets', metavar='N', type=str, nargs='+', choices=['celebA', 'lsun', 'mnist','f-mnist'],
           help='name of dataset to download [celebA, lsun, mnist, fmnist]')

def download(url, dirpath):
  filename = url.split('/')[-1]
  filepath = os.path.join(dirpath, filename)
  u = urllib.request.urlopen(url)
  f = open(filepath, 'wb')
  filesize = int(u.headers["Content-Length"])
  print("Downloading: %s Bytes: %s" % (filename, filesize))

  downloaded = 0
  block_sz = 8192
  status_width = 70
  while True:
    buf = u.read(block_sz)
    if not buf:
      print('')
      break
    else:
      print('', end='\r')
    downloaded += len(buf)
    f.write(buf)
    status = (("[%-" + str(status_width + 1) + "s] %3.2f%%") %
      ('=' * int(float(downloaded) / filesize * status_width) + '>', downloaded * 100. / filesize))
    print(status, end='')
    sys.stdout.flush()
  f.close()
  return filepath

def download_file_from_google_drive(id, destination):
  URL = "https://docs.google.com/uc?export=download"
  session = requests.Session()

  response = session.get(URL, params={ 'id': id }, stream=True,verify=False)
  token = get_confirm_token(response)

  if token:
    params = { 'id' : id, 'confirm' : token }
    response = session.get(URL, params=params, stream=True,verify=False)

  save_response_content(response, destination)

def get_confirm_token(response):
  for key, value in response.cookies.items():
    if key.startswith('download_warning'):
      return value
  return None

def save_response_content(response, destination, chunk_size=32*1024):
  total_size = int(response.headers.get('content-length', 0))
  with open(destination, "wb") as f:
    for chunk in tqdm(response.iter_content(chunk_size), total=total_size,
              unit='B', unit_scale=True, desc=destination):
      if chunk: # filter out keep-alive new chunks
        f.write(chunk)

def unzip(filepath):
  print("Extracting: " + filepath)
  dirpath = os.path.dirname(filepath)
  with zipfile.ZipFile(filepath) as zf:
    zf.extractall(dirpath)
  os.remove(filepath)

def download_celeb_a(dirpath):
  data_dir = 'celebA'
  if os.path.exists(os.path.join(dirpath, data_dir)):
    print('Found Celeb-A - skip')
    return

  filename, drive_id  = "img_align_celeba.zip", "0B7EVK8r0v71pZjFTYXZWM3FlRnM"
  save_path = os.path.join(dirpath, filename)

  if os.path.exists(save_path):
    print('[*] {} already exists'.format(save_path))
  else:
    download_file_from_google_drive(drive_id, save_path)

  zip_dir = ''
  with zipfile.ZipFile(save_path) as zf:
    zip_dir = zf.namelist()[0]
    zf.extractall(dirpath)
  os.remove(save_path)
  os.rename(os.path.join(dirpath, zip_dir), os.path.join(dirpath, data_dir))

def _list_categories(tag):
  url = 'http://lsun.cs.princeton.edu/htbin/list.cgi?tag=' + tag
  f = urllib.request.urlopen(url)
  return json.loads(f.read())

def _download_lsun(out_dir, category, set_name, tag):
  url = 'http://lsun.cs.princeton.edu/htbin/download.cgi?tag={tag}' \
      '&category={category}&set={set_name}'.format(**locals())
  print(url)
  if set_name == 'test':
    out_name = 'test_lmdb.zip'
  else:
    out_name = '{category}_{set_name}_lmdb.zip'.format(**locals())
  out_path = os.path.join(out_dir, out_name)
  cmd = ['curl', url, '-o', out_path]
  print('Downloading', category, set_name, 'set')
  subprocess.call(cmd)

def download_lsun(dirpath):
  data_dir = os.path.join(dirpath, 'lsun')
  if os.path.exists(data_dir):
    print('Found LSUN - skip')
    return
  else:
    os.mkdir(data_dir)

  tag = 'latest'
  categories = ['bedroom']

  for category in categories:
    _download_lsun(data_dir, category, 'train', tag)
    _download_lsun(data_dir, category, 'val', tag)
  _download_lsun(data_dir, '', 'test', tag)

def download_mnist(dirpath):
  data_dir = os.path.join(dirpath, 'mnist')
  if os.path.exists(data_dir):
    print('Found MNIST - skip')
    return
  else:
    os.mkdir(data_dir)
  url_base = 'http://yann.lecun.com/exdb/mnist/'
  file_names = ['train-images-idx3-ubyte.gz',
                'train-labels-idx1-ubyte.gz',
                't10k-images-idx3-ubyte.gz',
                't10k-labels-idx1-ubyte.gz']
  for file_name in file_names:
    url = (url_base+file_name).format(**locals())
    print(url)
    out_path = os.path.join(data_dir,file_name)
    cmd = ['curl', url, '-o', out_path]
    print('Downloading ', file_name)
    subprocess.call(cmd)
    cmd = ['gzip', '-d', out_path]
    print('Decompressing ', file_name)
    subprocess.call(cmd)


def download_fmnist(dirpath):
  data_dir = os.path.join(dirpath, 'f-mnist')
  if os.path.exists(data_dir):
    print('Found F-MNIST - skip')
    return
  else:
    os.mkdir(data_dir)
  url_base = 'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/'
  file_names = ['train-images-idx3-ubyte.gz',
                'train-labels-idx1-ubyte.gz',
                't10k-images-idx3-ubyte.gz',
                't10k-labels-idx1-ubyte.gz']
  for file_name in file_names:
    url = (url_base+file_name).format(**locals())
    print(url)
    out_path = os.path.join(data_dir,file_name)
    cmd = ['curl', url, '-o', out_path]
    print('Downloading ', file_name)
    subprocess.call(cmd)
    cmd = ['gzip', '-d', out_path]
    print('Decompressing ', file_name)
    subprocess.call(cmd)

def prepare_data_dir(path = './data'):
  if not os.path.exists(path):
    os.mkdir(path)

In [6]:
  prepare_data_dir()

  if any(name in DATASETS for name in ['CelebA', 'celebA', 'celebA']):
    download_celeb_a('./data')
  if 'mnist' in DATASETS:
    download_mnist('./data')
  if 'fmnist' in DATASETS:
    download_fmnist('./data')

http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
Decompressing  train-images-idx3-ubyte.gz
http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
Decompressing  train-labels-idx1-ubyte.gz
http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
Decompressing  t10k-images-idx3-ubyte.gz
http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
Decompressing  t10k-labels-idx1-ubyte.gz


In [7]:
def main(cfg, *args):
    FLAGS = tf.app.flags.FLAGS
    ds_gan = {
        'mnist': MnistDefenseGAN, 'f-mnist': FmnistDefenseDefenseGAN,
        'celeba': CelebADefenseGAN,
    }
    GAN = ds_gan[FLAGS.dataset_name]

    gan = GAN(cfg=cfg, test_mode=not FLAGS.is_train)

    if FLAGS.is_train:
        gan.train()

    if FLAGS.train_encoder:
        gan.load(checkpoint_dir=FLAGS.init_path)
        gan.train(phase='just_enc')

    if FLAGS.save_recs:
        gan.reconstruct_dataset(ckpt_path=FLAGS.init_path,
                                max_num=FLAGS.max_num)

    if FLAGS.test_generator:
        gan.load_generator(ckpt_path=FLAGS.init_path)
        gan.sess.run(gan.global_step.initializer)
        gan.generate_image(iteration=0)

    if FLAGS.test_batch:
        gan.test_batch()

    if FLAGS.save_ds:
        gan.save_ds()


In [8]:
    # Note: The load_config() call will convert all the parameters that are defined in
    # experiments/config files into FLAGS.param_name and can be passed in from command line.
    # arguments : python train.py --cfg <config_path> --<param_name> <param_value>
    cfg = load_config(CONFIG_FILE)
    flags = tf.app.flags

    flags.DEFINE_boolean("is_train", False,
                         "True for training, False for testing. [False]")
    flags.DEFINE_boolean("save_recs", False,
                         "True for saving reconstructions. [False]")
    flags.DEFINE_boolean("debug", False,
                         "True for debug. [False]")
    flags.DEFINE_boolean("test_generator", False,
                         "True for generator samples. [False]")
    flags.DEFINE_boolean("test_decoder", False,
                         "True for decoder samples. [False]")
    flags.DEFINE_boolean("test_again", False,
                         "True for not using cache. [False]")
    flags.DEFINE_boolean("test_batch", False,
                         "True for visualizing the batches and labels. [False]")
    flags.DEFINE_boolean("save_ds", False,
                         "True for saving the dataset in a pickle file. ["
                         "False]")
    flags.DEFINE_boolean("tensorboard_log", True, "True for saving "
                                                  "tensorboard logs. [True]")
    flags.DEFINE_boolean("train_encoder", False,
                         "Add an encoder to a pretrained model. ["
                         "False]")
    flags.DEFINE_boolean("init_with_enc", False,
                         "Initializes the z with an encoder, must run "
                         "--train_encoder first. [False]")
    flags.DEFINE_integer("max_num", -1,
                         "True for saving the dataset in a pickle file ["
                         "False]")
    flags.DEFINE_string("init_path", None, "Checkpoint path. [None]")


In [9]:
%tb

main_cfg = lambda x: main(cfg, x)
tf.app.run(main=main_cfg)


[#] MnistDefenseGAN.dataset_name is set to mnist.
[#] MnistDefenseGAN.batch_size is set to 50.
[#] MnistDefenseGAN.use_bn is set to False.
[#] MnistDefenseGAN.test_batch_size is set to 20.
[#] MnistDefenseGAN.mode is set to wgan-gp.
[#] MnistDefenseGAN.gradient_penalty_lambda is set to 10.0.
[#] MnistDefenseGAN.train_iters is set to 200000.
[#] MnistDefenseGAN.critic_iters is set to 5.
[#] MnistDefenseGAN.latent_dim is set to 128.
[#] MnistDefenseGAN.net_dim is set to 64.
[#] MnistDefenseGAN.input_transform_type is set to 0.
[#] MnistDefenseGAN.debug is set to False.
[#] MnistDefenseGAN.rec_iters is set to 200.
[#] MnistDefenseGAN.image_dim is set to [28, 28, 1].
[#] MnistDefenseGAN.rec_rr is set to 10.
[#] MnistDefenseGAN.rec_lr is set to 10.0.
[#] MnistDefenseGAN.test_again is set to False.
[-] MnistDefenseGAN.loss_type is not set.
[#] MnistDefenseGAN.loss_type is set to None.
[-] MnistDefenseGAN.attribute is not set.
[#] MnistDefenseGAN.attribute is set to None.
[#] MnistDefenseGAN.

No traceback available to show.
W1015 19:21:07.776894 140338511050624 module_wrapper.py:139] From models/base_model.py:67: The name tf.train.get_or_create_global_step is deprecated. Please use tf.compat.v1.train.get_or_create_global_step instead.



W1015 19:21:07.786226 140338511050624 module_wrapper.py:139] From models/base_model.py:69: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



W1015 19:21:07.794621 140338511050624 module_wrapper.py:139] From models/base_model.py:72: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



[+] Created the directory: output/gans/mnist
[+] Created the directory: debug/gans/mnist



...

W1015 19:21:07.954972 140338511050624 module_wrapper.py:139] From models/gan.py:123: The name tf.get_variable_scope is deprecated. Please use tf.compat.v1.get_variable_scope instead.



W1015 19:21:08.043596 140338511050624 module_wrapper.py:139] From models/base_model.py:243: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



W1015 19:21:08.058140 140338511050624 module_wrapper.py:139] From models/gan.py:85: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.



SystemExit: ignored

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
